In [77]:
import requests

url = "http://127.0.0.1:8777/api/tools/call"
headers = {"Content-Type": "application/json", "X-AdaOS-Token": "dev-local-token"}  # подставь актуальный токен
payload = {"tool": "weather_skill:get_weather", "arguments": {"city": "Berlin"}}

response = requests.post(url, json=payload, headers=headers)

print("Status:", response.status_code)
try:
    print("Response:", response.json())
except ValueError:
    print("Raw Response:", response.text)

Status: 200
Response: {'ok': True, 'result': {'ok': True, 'city': 'Berlin', 'temp': 24.2, 'description': 'few clouds'}, 'trace_id': '296b5b53-8ac5-4d40-a7af-f3d7000a8763'}


In [19]:
from adaos.sdk.subnet import subnet_set, subnet_get

subnet_set("home.city", "Berlin")
print(subnet_get("home.city"))

Berlin


In [47]:
!curl -i http://127.0.0.1:8777/health/live

HTTP/1.1 200 OK
date: Wed, 20 Aug 2025 11:21:36 GMT
server: uvicorn
content-length: 11
content-type: application/json

{"ok":true}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    11  100    11    0     0   3955      0 --:--:-- --:--:-- --:--:--  5500


In [7]:
!curl -H "X-AdaOS-Token: dev-local-token" http://127.0.0.1:8777/api/subnet/nodes


{"ok":true,"nodes":[{"node_id":"fe3d776d-dfcf-4eb4-9b2c-7a218f457c07","last_seen":1756823492.1923327,"status":"up","meta":{"hostname":"INIMATIC","roles":["member"]}}]}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   167  100   167    0     0  34676      0 --:--:-- --:--:-- --:--:-- 41750


In [8]:
print(requests.get("http://127.0.0.1:8777/api/node/status", headers=headers).json())

{'node_id': 'e7f6d431-9a32-4474-89f1-da6aac89ed9b', 'subnet_id': 'eb20131a-a2d6-48bb-b0ba-82add9bdd294', 'role': 'hub', 'hub_url': 'http://127.0.0.1:8777', 'ready': True}


In [6]:
import requests

headers = {"X-AdaOS-Token": "dev-local-token"}

# 1) проверить статус
print(requests.get("http://127.0.0.1:8778/api/node/status", headers=headers).json())

# 2) сменить роль на member или hub (и задать hub_url)
payload = {"role": "member", "hub_url": "http://127.0.0.1:8777"}
print(requests.post("http://127.0.0.1:8778/api/node/role", json=payload, headers=headers).json())

# 3) снова статус — должен быть role=member, ready=true
print(requests.get("http://127.0.0.1:8778/api/node/status", headers=headers).json())

{'node_id': 'fe3d776d-dfcf-4eb4-9b2c-7a218f457c07', 'subnet_id': 'eb20131a-a2d6-48bb-b0ba-82add9bdd294', 'role': 'member', 'hub_url': 'http://127.0.0.1:8777', 'ready': True}
{'ok': True, 'node': {'node_id': 'fe3d776d-dfcf-4eb4-9b2c-7a218f457c07', 'subnet_id': 'eb20131a-a2d6-48bb-b0ba-82add9bdd294', 'role': 'member', 'hub_url': 'http://127.0.0.1:8777', 'ready': True}, 'diagnostics': {'requested_role': 'member', 'hub_url': 'http://127.0.0.1:8777', 'subnet_id_used': 'eb20131a-a2d6-48bb-b0ba-82add9bdd294', 'now_ready': True}}
{'node_id': 'fe3d776d-dfcf-4eb4-9b2c-7a218f457c07', 'subnet_id': 'eb20131a-a2d6-48bb-b0ba-82add9bdd294', 'role': 'member', 'hub_url': 'http://127.0.0.1:8777', 'ready': True}


In [65]:
import requests

headers = {"X-AdaOS-Token": "dev-local-token"}
hub_status = requests.get("http://127.0.0.1:8777/api/node/status", headers=headers).json()
print(hub_status["subnet_id"])

eb20131a-a2d6-48bb-b0ba-82add9bdd294


In [ ]:
payload = {"role": "hub", "hub_url": "http://127.0.0.1:8777", "subnet_id": hub_status["subnet_id"]}
print(requests.post("http://127.0.0.1:8778/api/node/role", json=payload, headers=headers).json())

{'ok': True, 'node': {'node_id': 'fe3d776d-dfcf-4eb4-9b2c-7a218f457c07', 'subnet_id': 'eb20131a-a2d6-48bb-b0ba-82add9bdd294', 'role': 'member', 'hub_url': 'http://127.0.0.1:8777', 'ready': True}}


In [88]:
!curl -H "X-AdaOS-Token: dev-local-token" http://127.0.0.1:8777/api/subnet/nodes


{"ok":true,"nodes":[{"node_id":"fe3d776d-dfcf-4eb4-9b2c-7a218f457c07","last_seen":1755760798.9320548,"status":"up","meta":{"hostname":"INIMATIC","roles":["member"]}}]}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   167  100   167    0     0  41052      0 --:--:-- --:--:-- --:--:-- 41750


In [87]:
# AdaOS Observability v0 — sanity check notebook
#
# How to use:
# 1) Set HUB_URL, MEMBER_URL, TOKEN below for your setup.
# 2) Run this single cell. It will:
#    - ping health/live & health/ready
#    - show node status for hub & member
#    - list subnet nodes from hub
#    - (optionally) call a tool to generate events (TOOL_CALL_ENABLED)
#    - tail last N lines from hub /api/observe/tail
#
# All steps are guarded; failures are shown but won't stop the run.

import os, json, time, textwrap, pathlib, sys
from typing import Optional, Dict, Any, List

import requests

# ----------------- CONFIG -----------------
HUB_URL = os.environ.get("ADAOS_HUB_URL", "http://127.0.0.1:8777")
MEMBER_URL = os.environ.get("ADAOS_MEMBER_URL", "http://127.0.0.1:8778")
TOKEN = os.environ.get("ADAOS_TOKEN", "dev-local-token")

# If you have a working tool (e.g., weather_skill:get_weather) set this True
TOOL_CALL_ENABLED = True
TOOL_SPEC = "weather_skill:get_weather"
TOOL_ARGS = {"city": "Berlin"}

TAIL_LINES = 120
TOPIC_FILTER = None  # e.g. "ui." or "net.subnet." or None

# ------------------------------------------

HEADERS = {"X-AdaOS-Token": TOKEN, "Content-Type": "application/json"}


def hr(title: str):
    print("\n" + "=" * 20 + f" {title} " + "=" * 20)


def req_json(method: str, base: str, path: str, **kwargs) -> Dict[str, Any]:
    url = base.rstrip("/") + path
    try:
        r = requests.request(method, url, headers=HEADERS, timeout=5, **kwargs)
        try:
            data = r.json()
        except Exception:
            data = {"_raw": r.text}
        return {"ok": r.ok, "status": r.status_code, "url": url, "data": data, "headers": dict(r.headers)}
    except Exception as e:
        return {"ok": False, "status": None, "url": url, "error": str(e)}


def pretty_print(label: str, payload: Any):
    print(f"\n{label}:")
    if isinstance(payload, (dict, list)):
        print(json.dumps(payload, ensure_ascii=False, indent=2))
    else:
        print(payload)


# 1) HEALTH
hr("HEALTH (hub & member)")
hub_live = req_json("GET", HUB_URL, "/health/live")
hub_ready = req_json("GET", HUB_URL, "/health/ready")
mem_live = req_json("GET", MEMBER_URL, "/health/live")
mem_ready = req_json("GET", MEMBER_URL, "/health/ready")

pretty_print("hub/live", hub_live)
pretty_print("hub/ready", hub_ready)
pretty_print("member/live", mem_live)
pretty_print("member/ready", mem_ready)

# 2) NODE STATUS
hr("NODE STATUS (hub & member)")
hub_status = req_json("GET", HUB_URL, "/api/node/status")
mem_status = req_json("GET", MEMBER_URL, "/api/node/status")
pretty_print("hub status", hub_status)
pretty_print("member status", mem_status)

# 3) SUBNET NODES (from hub)
hr("SUBNET /api/subnet/nodes")
nodes = req_json("GET", HUB_URL, "/api/subnet/nodes")
pretty_print("nodes", nodes)

# 4) (Optional) TOOL CALL to generate events & trace
if TOOL_CALL_ENABLED:
    hr(f"TOOLS CALL → {TOOL_SPEC}")
    tool_resp = req_json("POST", MEMBER_URL, "/api/tools/call", json={"tool": TOOL_SPEC, "arguments": TOOL_ARGS})
    pretty_print("tool call response", tool_resp)
else:
    hr("TOOLS CALL skipped (TOOL_CALL_ENABLED=False)")

# 5) OBSERVE TAIL from hub
hr(f"OBSERVE TAIL last {TAIL_LINES} lines (hub)")
tail = req_json("GET", HUB_URL, f"/api/observe/tail?lines={TAIL_LINES}")
lines = []
if tail.get("ok") and isinstance(tail.get("data"), dict):
    lines = tail["data"].get("lines") or []
    # Optional filter by topic prefix
    if TOPIC_FILTER:
        keep = []
        for ln in lines:
            try:
                obj = json.loads(ln)
                if str(obj.get("topic", "")).startswith(TOPIC_FILTER):
                    keep.append(ln)
            except Exception:
                # keep raw if parsing fails and no filter
                pass
        lines = keep

print(f"tail lines: {len(lines)}")
for ln in lines:
    print(ln)

# 6) SUMMARY
hr("SUMMARY")
summary = {
    "hub_ready": hub_ready.get("ok") and hub_ready.get("status") == 200,
    "member_ready": mem_ready.get("ok") and mem_ready.get("status") == 200,
    "nodes_seen_on_hub": len(nodes.get("data", {}).get("nodes", [])) if isinstance(nodes.get("data"), dict) else None,
    "tool_call_status": hub_ready.get("status") if not TOOL_CALL_ENABLED else (tool_resp.get("status")),
    "tail_lines": len(lines),
}
pretty_print("summary", summary)


==================== HEALTH (hub & member) ====================

hub/live:
{
  "ok": true,
  "status": 200,
  "url": "http://127.0.0.1:8777/health/live",
  "data": {
    "ok": true
  },
  "headers": {
    "Connection": "close",
    "Content-Length": "11",
    "Content-Type": "application/json",
    "Date": "Thu, 21 Aug 2025 06:28:45 GMT",
    "Server": "uvicorn"
  }
}

hub/ready:
{
  "ok": true,
  "status": 200,
  "url": "http://127.0.0.1:8777/health/ready",
  "data": {
    "ok": true
  },
  "headers": {
    "Connection": "close",
    "Content-Length": "11",
    "Content-Type": "application/json",
    "Date": "Thu, 21 Aug 2025 06:28:45 GMT",
    "Server": "uvicorn"
  }
}

member/live:
{
  "ok": true,
  "status": 200,
  "url": "http://127.0.0.1:8778/health/live",
  "data": {
    "ok": true
  },
  "headers": {
    "Connection": "close",
    "Content-Length": "11",
    "Content-Type": "application/json",
    "Date": "Thu, 21 Aug 2025 06:28:45 GMT",
    "Server": "uvicorn"
  }
}

member/r

In [86]:
import os, json, time, requests

HUB_URL = os.environ.get("ADAOS_HUB_URL", "http://127.0.0.1:8777")
MEMBER_URL = os.environ.get("ADAOS_MEMBER_URL", "http://127.0.0.1:8778")
TOKEN = os.environ.get("ADAOS_TOKEN", "dev-local-token")

HEADERS = {"X-AdaOS-Token": TOKEN, "Content-Type": "application/json"}


def call(method, base, path, **kw):
    url = base.rstrip("/") + path
    try:
        r = requests.request(method, url, headers=HEADERS, timeout=5, **kw)
        try:
            data = r.json()
        except Exception:
            data = {"_raw": r.text}
        return r.status_code, data
    except Exception as e:
        return None, {"error": str(e)}


print("health hub:", call("GET", HUB_URL, "/health/ready"))
print("health member:", call("GET", MEMBER_URL, "/health/ready"))
print("hub status:", call("GET", HUB_URL, "/api/node/status"))
print("member status:", call("GET", MEMBER_URL, "/api/node/status"))
print("hub nodes:", call("GET", HUB_URL, "/api/subnet/nodes"))

# генерируем тестовое событие на member → оно попадёт в локальный лог и уйдёт батчем на hub
print("emit test (member):", call("POST", MEMBER_URL, "/api/observe/test", json={"kind": "ping", "note": "from-notebook"}))

# даём время батчеру переслать
time.sleep(2.0)

# хвост логов с hub
status, tail = call("GET", HUB_URL, "/api/observe/tail?lines=1000")
lines = tail.get("lines") or []
print("tail lines:", len(lines))

# фильтруем по нашему тестовому топику
obs = []
for ln in lines:
    try:
        evt = json.loads(ln)
        if evt.get("topic") == "obs.test.ping":
            obs.append(evt)
    except Exception:
        pass

print("found obs.test.ping:", len(obs))
if obs:
    print(json.dumps(obs[-1], ensure_ascii=False, indent=2))

health hub: (200, {'ok': True})
health member: (200, {'ok': True})
hub status: (200, {'node_id': 'e7f6d431-9a32-4474-89f1-da6aac89ed9b', 'subnet_id': 'eb20131a-a2d6-48bb-b0ba-82add9bdd294', 'role': 'hub', 'hub_url': 'http://127.0.0.1:8777', 'ready': True})
member status: (200, {'node_id': 'fe3d776d-dfcf-4eb4-9b2c-7a218f457c07', 'subnet_id': 'eb20131a-a2d6-48bb-b0ba-82add9bdd294', 'role': 'member', 'hub_url': 'http://127.0.0.1:8777', 'ready': True})
hub nodes: (200, {'ok': True, 'nodes': [{'node_id': 'fe3d776d-dfcf-4eb4-9b2c-7a218f457c07', 'last_seen': 1755757578.5121078, 'status': 'up', 'meta': {'hostname': 'INIMATIC', 'roles': ['member']}}]})
emit test (member): (200, {'ok': True, 'topic': 'obs.test.ping', 'payload': {'note': 'hello', 'at': 1755757581.113394}})
tail lines: 17
found obs.test.ping: 1
{
  "ts": 1755757581.113394,
  "topic": "obs.test.ping",
  "payload": {
    "note": "hello",
    "at": 1755757581.113394
  },
  "trace": "c9919127-9994-4da8-a249-26a8ec5ba039",
  "source": 

In [93]:
!curl -H "X-AdaOS-Token: dev-local-token" -X POST http://127.0.0.1:8777/api/observe/test

{"ok":true,"topic":"obs.test.ping","payload":{"note":"hello","at":1755762890.3656685}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    86  100    86    0     0   7972      0 --:--:-- --:--:-- --:--:--  8600


In [ ]:
from pathlib import Path

IMPORTANT = {"src", "adaos", "tests", "docs"}


def list_clean_tree(root="."):
    root_path = Path(root).resolve()

    def walk(path: Path, depth=0, max_depth=4):
        if depth > max_depth:
            return
        for p in sorted(path.iterdir()):
            if p.is_dir() and not p.name.startswith((".", "__")) and "egg-info" not in p.name:
                rel = p.relative_to(root_path)
                if any(part in IMPORTANT for part in rel.parts) or depth == 0:
                    print("  " * depth + f"- {rel}")
                walk(p, depth + 1, max_depth)

    print(f"Clean project tree at {root_path}:")
    walk(root_path)


if __name__ == "__main__":
    list_clean_tree("C:/git/MUIV/adaos")

Clean project tree at C:\git\MUIV\adaos:
- build
- docs
  - docs\diagrams
    - docs\diagrams\out
    - docs\diagrams\src
- src
  - src\adaos
    - src\adaos\abi
    - src\adaos\agent
      - src\adaos\agent\audio
        - src\adaos\agent\audio\old
        - src\adaos\agent\audio\stt
        - src\adaos\agent\audio\tts
      - src\adaos\agent\core
      - src\adaos\agent\db
      - src\adaos\agent\ovos_adapter
      - src\adaos\agent\platform
        - src\adaos\agent\platform\android
      - src\adaos\agent\utils
    - src\adaos\api
    - src\adaos\common
    - src\adaos\devportal
    - src\adaos\integrations
      - src\adaos\integrations\inimatic
        - src\adaos\integrations\inimatic\backend
        - src\adaos\integrations\inimatic\browser_extension
        - src\adaos\integrations\inimatic\deployment
        - src\adaos\integrations\inimatic\LoginWV
        - src\adaos\integrations\inimatic\resources
        - src\adaos\integrations\inimatic\src
        - src\adaos\integratio

# Scenarios

In [ ]:
import requests

H = {"X-AdaOS-Token": "dev-local-token"}

In [ ]:
r = requests.post(
    "http://127.0.0.1:8777/api/scenarios/create",
    json={"id": "introduction", "template": "template"},
    headers=H,
)
print(r.status_code, r.text)

200 {"ok":true,"path":"C:\\git\\MUIV\\adaos\\.adaos\\scenarios\\introduction\\scenario.json"}


In [ ]:
# установить сценарий из репо
requests.post("http://127.0.0.1:8777/api/scenarios/update_repo/morning", json={"ref": "main"}, headers=H).json()

{'ok': True,
 'path': 'C:\\git\\MUIV\\adaos\\.adaos\\scenarios\\morning\\scenario.json'}

In [86]:
# пуш изменений сценария
requests.post("http://127.0.0.1:8777/api/scenarios/install/morning", json={"message": "tune morning flow"}, headers=H).json()

{'ok': True,
 'message': "[green]scenario 'morning' pulled[/green] (version: 0.1.0)"}

In [ ]:
# пуш изменений сценария
requests.post("http://127.0.0.1:8777/api/scenarios/push/morning", json={"message": "tune morning flow"}, headers=H).json()

{'ok': True,
 'message': "[green]scenario 'morning' pushed[/green] (version: 0.1.0)"}

In [85]:
requests.post("http://127.0.0.1:8777/api/scenarios/uninstall/morning", headers=H).json()

{'ok': True, 'message': "[green]scenario 'morning' uninstalled[/green]"}

In [87]:
requests.get("http://127.0.0.1:8777/api/scenarios/list", headers=H).json()

{'items': [{'id': 'template-id',
   'name': 'New Scenario',
   'version': '0.1.0',
   'path': 'C:\\git\\MUIV\\adaos\\.adaos\\scenarios\\morning\\scenario.json',
   'meta': None}]}

In [ ]:
# run in project root: python - <<'PY'
from adaos.services.settings import Settings
from adaos.adapters.fs.path_provider import LocalPathProvider
from adaos.services.agent_context import AgentContext
from adaos.ports import EventBus, Process, Capabilities, Devices, KV, SQL, Secrets, Net, Updates


class Dummy:
    pass


ctx = AgentContext(
    settings=Settings.from_env(),
    # в PR-1 поля paths ещё нет; после применения PR-2 — раскомментируйте:
    # paths=LocalPathProvider(Settings.from_env()),
    bus=Dummy(),
    proc=Dummy(),
    caps=Dummy(),
    devices=Dummy(),
    kv=Dummy(),
    sql=Dummy(),
    secrets=Dummy(),
    net=Dummy(),
    updates=Dummy(),
)
print("OK: Settings base_dir =", ctx.settings.base_dir)
# print("OK: skills_dir =", ctx.paths.skills_dir())  # работает после PR-2
# PY

OK: Settings base_dir = C:\Users\Zver/.adaos
